<a href="https://colab.research.google.com/github/WestonMadeira1/HW_5/blob/main/HW5P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import pandas as pd
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file_path = '/content/drive/My Drive/ML Data/Housing.csv'
dataset = pd.read_csv(file_path)
dataset.head

<bound method NDFrame.head of         price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0    13300000  7420         4          2        3      yes        no       no   
1    12250000  8960         4          4        4      yes        no       no   
2    12250000  9960         3          2        2      yes        no      yes   
3    12215000  7500         4          2        2      yes        no      yes   
4    11410000  7420         4          1        2      yes       yes      yes   
..        ...   ...       ...        ...      ...      ...       ...      ...   
540   1820000  3000         2          1        1      yes        no      yes   
541   1767150  2400         3          1        1       no        no       no   
542   1750000  3620         2          1        1      yes        no       no   
543   1750000  2910         3          1        1       no        no       no   
544   1750000  3850         3          1        2      yes        no       no  

In [20]:
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

def binary_map(x):
    return x.map({'yes': 1, 'no': 0})


In [21]:
dataset[varlist] = dataset[varlist].apply(binary_map)
dataset.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,4.566365,1.046726,1.403419,1.421812,1.378217,NaN,NaN,NaN,NaN,NaN,1.517692,NaN,NaN
1,4.004484,1.757010,1.403419,5.405809,2.532024,NaN,NaN,NaN,NaN,NaN,2.679409,NaN,NaN
2,4.004484,2.218232,0.047278,1.421812,0.224410,NaN,NaN,NaN,NaN,NaN,1.517692,NaN,NaN
3,3.985755,1.083624,1.403419,1.421812,0.224410,NaN,NaN,NaN,NaN,NaN,2.679409,NaN,NaN
4,3.554979,1.046726,1.403419,-0.570187,0.224410,NaN,NaN,NaN,NaN,NaN,1.517692,NaN,NaN


In [22]:
from sklearn.preprocessing import StandardScaler
import statistics
# Calculate the standard deviation
columns = ['price','area','bedrooms','bathrooms','stories','mainroad','guestroom','basement','hotwaterheating','airconditioning','parking','prefarea','furnishingstatus']

scaler = StandardScaler()

# Fit and transform the scaler into the forementioned calculation
dataset[columns] = scaler.fit_transform(dataset[columns])
dataset.head()

/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,4.566365,1.046726,1.403419,1.421812,1.378217,NaN,NaN,NaN,NaN,NaN,1.517692,NaN,NaN
1,4.004484,1.757010,1.403419,5.405809,2.532024,NaN,NaN,NaN,NaN,NaN,2.679409,NaN,NaN
2,4.004484,2.218232,0.047278,1.421812,0.224410,NaN,NaN,NaN,NaN,NaN,1.517692,NaN,NaN
3,3.985755,1.083624,1.403419,1.421812,0.224410,NaN,NaN,NaN,NaN,NaN,2.679409,NaN,NaN
4,3.554979,1.046726,1.403419,-0.570187,0.224410,NaN,NaN,NaN,NaN,NaN,1.517692,NaN,NaN


In [23]:
X = dataset.iloc[:, [1,2,3,4,10]].values
Y = dataset.iloc[:, 0].values

X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32).view(-1, 1)

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_tensor, Y_tensor, test_size=0.2, random_state=0)

In [25]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_size):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, 1)

    def forward(self, x):
        return self.linear(x)

In [26]:
def training_loop(X_train, Y_train, X_val, Y_val, learning_rate, num_epochs=5000):
    input_size = X_train.shape[1]
    model = LinearRegressionModel(input_size)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        outputs = model(X_train)
        loss = criterion(outputs, Y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 500 == 0:
            train_losses.append(loss.item())
            with torch.no_grad():
                model.eval()
                val_outputs = model(X_val)
                val_loss = criterion(val_outputs, Y_val)
                val_losses.append(val_loss.item())

                print('Epoch [{}/{}], Training Loss: {:.4f}, Validation Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item(), val_loss.item()))

            model.train()

    return model, train_losses, val_losses

In [27]:
learning_rates = [0.1, 0.01, 0.001, 0.0001]

best_model = None
best_val_loss = float('inf')
best_lr = None

all_train_losses = []
all_val_losses = []

for lr in learning_rates:
    print('\Llearning rate: {}'.format(lr))
    model, train_losses, val_losses = training_loop(X_train, Y_train, X_val, Y_val, lr)

    # Pick the best model based on validation loss
    if val_losses[-1] < best_val_loss:
        best_val_loss = val_losses[-1]
        best_model = model
        best_lr = lr

    # Append the loss lists
    all_train_losses.append(train_losses)
    all_val_losses.append(val_losses)

\Llearning rate: 0.1
Epoch [500/5000], Training Loss: 0.4656, Validation Loss: 0.3361
Epoch [1000/5000], Training Loss: 0.4656, Validation Loss: 0.3361
Epoch [1500/5000], Training Loss: 0.4656, Validation Loss: 0.3361
Epoch [2000/5000], Training Loss: 0.4656, Validation Loss: 0.3361
Epoch [2500/5000], Training Loss: 0.4656, Validation Loss: 0.3361
Epoch [3000/5000], Training Loss: 0.4656, Validation Loss: 0.3361
Epoch [3500/5000], Training Loss: 0.4656, Validation Loss: 0.3361
Epoch [4000/5000], Training Loss: 0.4656, Validation Loss: 0.3361
Epoch [4500/5000], Training Loss: 0.4656, Validation Loss: 0.3361
Epoch [5000/5000], Training Loss: 0.4656, Validation Loss: 0.3361
\Llearning rate: 0.01
Epoch [500/5000], Training Loss: 0.4656, Validation Loss: 0.3360
Epoch [1000/5000], Training Loss: 0.4656, Validation Loss: 0.3361
Epoch [1500/5000], Training Loss: 0.4656, Validation Loss: 0.3361
Epoch [2000/5000], Training Loss: 0.4656, Validation Loss: 0.3361
Epoch [2500/5000], Training Loss: 0